# MODEL NN 01

In [18]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, SimpleRNN, LSTM, Dense, Dropout, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD
import warnings
warnings.filterwarnings('ignore')

# Define ticker symbol and data range
ticker = "BTC-USD"
start = "2000-01-01"
end = yf.Ticker(ticker).history(period="5d").index[0].strftime("%Y-%m-%d")

# Download historical data
df = yf.download(ticker, start=start, end=end)

# INDICATORS
rsi_indicator = RSIIndicator(df['Close'], window=14)
bb_indicator = BollingerBands(df['Close'], window=20)

# Feature engineering / number of features
def create_features(df):
    df['Change'] = df['Close'] - df['Open']
    df['Pct_Change'] = df['Change'] / df['Open'] * 100
    df['Moving_Avg_5'] = df['Close'].rolling(window=5).mean()
    df['Moving_Avg_10'] = df['Close'].rolling(window=10).mean()
    df['Moving_Avg_50'] = df['Close'].rolling(window=50).mean()
    df['Moving_Avg_200'] = df['Close'].rolling(window=200).mean()
    df['Exp_Moving_Avg'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['Bollinger_Bands'] = bb_indicator.bollinger_mavg()
    df['RSI'] = rsi_indicator.rsi()
    return df

df = create_features(df.copy())

# Remove rows with NaN values
df.dropna(inplace=True)

# Split data into features (X) and target (y)
X = df.drop(['Close'], axis=1)
y = df['Close']

# Impute missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_imputed)

# Check the number of features in X_scaled
n_samples = X_scaled.shape[0]
n_features = X_scaled.shape[1]

# Choose the number of timesteps, for example, 7 timesteps
n_timesteps = 7  # Increase timesteps to 7 for better temporal structure
n_features_per_timestep = X_scaled.shape[1] // n_timesteps

# Ensure the number of features is divisible by the number of timesteps
if n_features % n_timesteps != 0:
    raise ValueError(f"Number of features ({n_features}) is not divisible by timesteps ({n_timesteps})")

# Reshape X_scaled to (samples, timesteps, features_per_timestep)
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], n_timesteps, n_features_per_timestep))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Build CNN + RNN + LSTM hybrid model with batch normalization and no Flatten
model = Sequential()

# 1. CNN layer: Extracts features from time series
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())  # Added batch normalization after CNN

# 2. RNN layer: Captures short-term dependencies
model.add(SimpleRNN(units=32, return_sequences=True)) # 50
model.add(BatchNormalization())  # Batch normalization to stabilize RNN layer

# 3. LSTM layers: Learns long-term dependencies
model.add(LSTM(units=256, return_sequences=True))  # Increased LSTM units for better memory 128
model.add(Dropout(0.3))  # Increased dropout rate to avoid overfitting
model.add(LSTM(units=128)) #64
model.add(Dropout(0.3))

# 4. Output layer: Predict the closing price
model.add(Dense(units=1))

# Compile the model with Adam optimizer and a lower learning rate
# optimizer / Adam, RMSprop, Adagrad
optimizer = Adam(learning_rate=0.005) # 0.0005, 0.0001
model.compile(optimizer=optimizer, loss='mse')

# Add a learning rate scheduler to reduce the learning rate dynamically
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)

# Train the model
model.fit(X_train, np.reshape(y_train.values, (-1, 1)), epochs=50, batch_size=64, validation_data=(X_test, np.reshape(y_test.values, (-1, 1))), callbacks=[lr_scheduler])

# Make predictions
predictions = model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

[*********************100%***********************]  1 of 1 completed


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - loss: 793677568.0000 - val_loss: 715307008.0000 - learning_rate: 0.0050
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 805496192.0000 - val_loss: 714395648.0000 - learning_rate: 0.0050
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 764160896.0000 - val_loss: 713501376.0000 - learning_rate: 0.0050
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 819097664.0000 - val_loss: 712612416.0000 - learning_rate: 0.0050
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 800596864.0000 - val_loss: 711757824.0000 - learning_rate: 0.0050
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 796674304.0000 - val_loss: 710896128.0000 - learning_rate: 0.0050
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 772096000.0000 - val_loss: 710057088.0000 - learning_rate: 0.0050
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 768410624.0000 - val_loss: 709208832.0000 - learning_rate: 0.0050

# MODEL NN 02

In [23]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, SimpleRNN, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD
from scipy.stats import zscore
from datetime import datetime, timedelta

# Define ticker symbol and data range
ticker = "BTC-USD"
start = "2000-01-01"
end = (datetime.today() - timedelta(days=5)).strftime("%Y-%m-%d")

# Download historical data
df = yf.download(ticker, start=start, end=end)

# Preprocess the data
df['Date'] = pd.to_datetime(df.index)
df['Date'] = df['Date'].apply(lambda date: date.timestamp())

# Remove noise and outliers
z_scores = zscore(df[['Open', 'High', 'Low', 'Close', 'Volume']])
df = df[(z_scores < 3).all(axis=1)]

# Feature engineering
df['Price_Change_Pct'] = df['Close'].pct_change()
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df['50d_MA'] = df['Close'].rolling(window=50).mean()
df['200d_MA'] = df['Close'].rolling(window=200).mean()

# Technical indicators
rsi_indicator = RSIIndicator(df['Close'], window=14)
df['RSI'] = rsi_indicator.rsi()
bb_indicator = BollingerBands(df['Close'], window=20)
df['Bollinger_Bands'] = bb_indicator.bollinger_mavg()
macd_indicator = MACD(df['Close'], window_slow=26, window_fast=12)
df['MACD'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()

# Remove NA
df = df.dropna()

# Split the data into features and target
X = df[['Price_Change_Pct', 'Log_Returns', '50d_MA', '200d_MA', 'RSI', 'Bollinger_Bands', 'MACD']]
y = df['Close']

# Impute missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_imputed)

n_features = X_scaled.shape[1]

# Reshape X_scaled for CNN-RNN input
n_timesteps = 7  # Set timesteps based on sequence length
n_samples = (X_scaled.shape[0] // n_timesteps) * n_timesteps  # Ensure divisible by n_timesteps

# Truncate the data so it can be reshaped
X_scaled = X_scaled[:n_samples, :]  # Truncate to be divisible by n_timesteps
y = y.iloc[:n_samples]  # Truncate y to match X

# Reshape the data
X_scaled = X_scaled.reshape((n_samples // n_timesteps, n_timesteps, n_features))

# Ensure target shape matches the X input sequence
y = y.values.reshape((-1, n_timesteps, 1))  # Match the shape of target for each timestep

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Build the hybrid model
model = Sequential()

# 1. CNN layer: Extracts features from time series
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())  # Added batch normalization after CNN

# 2. RNN layer: Captures short-term dependencies
model.add(SimpleRNN(units=32, return_sequences=True))
model.add(BatchNormalization())  # Batch normalization to stabilize RNN layer

# 3. LSTM layers: Learns long-term dependencies
model.add(LSTM(units=256, return_sequences=True))  # Increased LSTM units for better memory
model.add(Dropout(0.3))  # Increased dropout rate to avoid overfitting
model.add(LSTM(units=128))
model.add(Dropout(0.3))

# 4. Output layer: Predict the closing price
model.add(Dense(units=1))

# Compile the model with Adam optimizer and a lower learning rate
optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)

# Since the model predicts one value per sequence, select only the last timestep from y_test
y_test_last = y_test[:, -1]  # Last value of each sequence

# Evaluate performance using the last timestep values
mse = mean_squared_error(y_test_last.flatten(), predictions.flatten())
print(f"Mean Squared Error: {mse}")

[*********************100%***********************]  1 of 1 completed


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 202ms/step - loss: 661631872.0000 - val_loss: 864700800.0000
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 727073792.0000 - val_loss: 864389312.0000
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 666940608.0000 - val_loss: 864233280.0000
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 733420416.0000 - val_loss: 864073280.0000
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 663612096.0000 - val_loss: 863913216.0000
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 735389440.0000 - val_loss: 863750528.0000
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 718261312.0000 - val_loss: 863590720.0000
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 665427712.0000 - val_loss: 863432320.0000
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 721227648.0000 - val_loss: 863270464.0000
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 678505600.0000 - val_loss: 863111552.0000
Epoch 1

# MODEL NN 03

In [22]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, SimpleRNN, LSTM, Dense, Dropout, Reshape, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD
from scipy.stats import zscore
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit

# Define ticker symbol and data range
ticker = "BTC-USD"
start = "2000-01-01"
end = (datetime.today() - timedelta(days=5)).strftime("%Y-%m-%d")

# Download historical data
df = yf.download(ticker, start=start, end=end)

# Preprocess the data
df['Date'] = pd.to_datetime(df.index)
df['Date'] = df['Date'].apply(lambda date: date.timestamp())

# Remove noise and outliers
z_scores = zscore(df[['Open', 'High', 'Low', 'Close', 'Volume']])
df = df[(z_scores < 3).all(axis=1)]

# Feature engineering
df['Price_Change_Pct'] = df['Close'].pct_change()
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
df['50d_MA'] = df['Close'].rolling(window=50).mean()
df['200d_MA'] = df['Close'].rolling(window=200).mean()

# Technical indicators
rsi_indicator = RSIIndicator(df['Close'], window=14)
df['RSI'] = rsi_indicator.rsi()
bb_indicator = BollingerBands(df['Close'], window=20)
df['Bollinger_Bands'] = bb_indicator.bollinger_mavg()
macd_indicator = MACD(df['Close'], window_slow=26, window_fast=12)
df['MACD'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()

# Remove NA
df = df.dropna()

# Split the data into features and target
X = df[['Price_Change_Pct', 'Log_Returns', '50d_MA', '200d_MA', 'RSI', 'Bollinger_Bands', 'MACD']]
y = df['Close']

# Impute missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_imputed)

# Reshape X_scaled to fit CNN input
n_timesteps = 10
n_features = X_scaled.shape[1]
X_scaled = np.reshape(X_scaled, (-1, n_timesteps, n_features))

# Split data into training and testing sets
tscv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Build the hybrid model
model = Sequential()

# 1. CNN layer: Extracts features from time series
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())

# 2. Bidirectional RNN layer: Captures past and future dependencies
model.add(Bidirectional(SimpleRNN(units=64, return_sequences=True)))
model.add(BatchNormalization())

# 3. LSTM layers: Learns long-term dependencies
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=128))
model.add(Dropout(0.3))

# 4. Output layer: Predict the closing price
model.add(Dense(units=1))

# Compile the model with Adam optimizer and a lower learning rate
optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Predictions
predictions = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

[*********************100%***********************]  1 of 1 completed


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 329ms/step - loss: 93930.2969 - val_loss: 164444.1719
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 85989.1797 - val_loss: 159156.6562
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 83020.3516 - val_loss: 156149.9375
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 80757.5234 - val_loss: 153465.6562
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 77485.8281 - val_loss: 150800.9688
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 75516.4766 - val_loss: 148161.4531
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 78992.1094 - val_loss: 145556.2188
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 73794.3672 - val_loss: 143012.8281
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 73517.9609 - val_loss: 140510.1250
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 69662.2188 - val_loss: 138061.7812
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 65424.5430 - val_lo